# Part 2 - Gathering Live Tweets
This notebook is the beginning of Part 2 which attempts to validate the findings in Part 1 on real-time conditions with data pulled from the Twitter API and the Yahoo Finance API. In the cell below, I import the necessary tools, connect to the Twitter API, write the function to calculate the sentiment scores of each tweet and define the stocks that I am performing the analysis on.  In this case, I am using the same stocks as in the previous notebooks with the exception of those companies who have been absorbed by other companies and thus no longer have a valid ticker.

In [1]:
import tweepy as tw
import pandas as pd
import datetime, time
import matplotlib.pyplot as plt
import pickle
from scipy import stats
pd.core.common.is_list_like=pd.api.types.is_list_like
import pandas_datareader as pdr
from textblob import TextBlob
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader=SentimentIntensityAnalyzer()
pd.core.common.is_list_like=pd.api.types.is_list_like
import pandas_datareader as pdr

twitter_api_key='TdSeHGzbz3V1MWGUUQaKSY1w2'
twitter_api_secret='nulKL65bYeUOmvkt6XuGsChCgKrWh9C0zJmJYPgeqlfcdWgvN2'
app_token='983746297135067137-uouYUhYF7uK56qiNu2sQ48PPLtwetvn'
app_secret='OxCAz0jgNwngT8kceI1PgbUYqmQgr0c8hRuwQ6dXhJ1p0'
auth=tw.OAuthHandler(twitter_api_key,twitter_api_secret)
auth.set_access_token(app_token,app_secret)
api=tw.API(auth,wait_on_rate_limit=True)

def add_sentiment_scores(df,ticker_sym):
    vader_score=[]
    weighted_vader=[]
    blob_score=[]
    weighted_blob=[]
    ticker=[]
    for i in range(len(df)):
        try:
            text=df['text'][i]
            follow=df.followers[i]
            vad=vader.polarity_scores(text)
            blb=TextBlob(text)
            vs=vad['compound']
            bl=blb.sentiment[0]
            vader_score.append(vs)
            weighted_vader.append(vs*follow)
            blob_score.append(bl)
            weighted_blob.append(bl*follow)
            ticker.append(ticker_sym)
        except:
            vader_score.append(0)
            weighted_vader.append(0)
            blob_score.append(0)
            weighted_blob.append(0)
    vader_score=pd.DataFrame(vader_score,columns=['vader_score'])
    weighted_vader=pd.DataFrame(weighted_vader,columns=['weighted_vader'])
    blob_score=pd.DataFrame(blob_score,columns=['blob_score'])
    weighted_blob=pd.DataFrame(weighted_blob,columns=['weighted_blob'])
    ticker=pd.DataFrame(ticker,columns=['ticker'])
    df=df.merge(vader_score,left_index=True,right_index=True)
    df=df.merge(weighted_vader,left_index=True,right_index=True)
    df=df.merge(blob_score,left_index=True,right_index=True)
    df=df.merge(weighted_blob,left_index=True,right_index=True)
    df=df.merge(ticker,left_index=True,right_index=True)
    return df

STOCKS=['aal','aapl','adbe','adp','adsk','akam','alxn','amat','amgn','amzn','atvi','bbby','bidu','bmrn',
        'celg','cern','chkp','chtr','cmcsa','cost','csco','csx','ctrp','dish','dltr','ea','ebay','endp',
        'expe','fast','fb','fisv','gild','goog','hsic','ilmn','incy','intu','isrg','jd','khc',
        'lrcx','mar','mat','mdlz','mnst','msft','mu','mxim','myl','nclh','nflx','ntap','nvda','nxpi',
        'orly','payx','pcar','pypl','qcom','regn','rost','sbac','sbux','srcl','stx','swks','symc',
        'tmus','trip','tsco','tsla','txn','ulta','vod','vrsk','vrtx','wba','wdc','xlnx']

There are several things to note:
- Depending on what Twitter API package you either pay or do not pay for, there are limitations on how much data and how often you can pull data.
- As far as I am able to tell, the free version only allows for tweets within the past week to be gathered.
- The commented line in the cell below should only be un-commented on the very first time you run the cell.  After that it will erase any data you have gathered.

In [3]:
#all_api_tweets=pd.DataFrame([])
start_date="2019-07-31"
for stock in STOCKS:
    tweet_holder=[]
    for tweet in tw.Cursor(api.search,q="$"+stock,count=1000000,lang="en",since=start_date).items():
        tweet_holder.append(tweet)
    tweet_df=pd.DataFrame([])
    for i in range(len(tweet_holder)):
        tweet_df=tweet_df.append(pd.DataFrame({'date_time':tweet_holder[i].created_at,
                                               'text':tweet_holder[i].text,
                                               'username':tweet_holder[i].user.screen_name,
                                               'followers':tweet_holder[i].user.followers_count},index=[0]),ignore_index=True)
    tweet_df.date_time=tweet_df.date_time-np.timedelta64(4,"h")
    tweet_df=add_sentiment_scores(tweet_df,stock)
    all_api_tweets=all_api_tweets.append(tweet_df)
all_api_tweets.reset_index(drop=True,inplace=True)
all_api_tweets['Date']=all_api_tweets.date_time.dt.strftime("%Y-%m-%d")
all_api_tweets['Date']=pd.to_datetime(all_api_tweets['Date'], infer_datetime_format=True)
pickle_out=open("all_api_tweets.pickle","wb")
pickle.dump(all_api_tweets,pickle_out)
pickle_out.close()

C:\Users\kylej\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


## Gathering Stock Movement Data
Here I first identify all of the stocks in questiona and all of the dates in which there are tweets and then find the each stock's movement on each day.  I then create a dataframe that has a row for each stock for each day as well as its various sentiment scores, the number of tweets and the resulting stock movement.  Lastly I calculate each stock's average tweet volume and the standard deviation of that tweet volume.

In [5]:
# gathers daily return for each stock on each day of a tweet
all_stocks=all_api_tweets.ticker.unique()
all_dates=all_api_tweets.Date.unique()
all_dates=all_dates[:-1]
daily_api_returns=pd.DataFrame(all_dates,columns=['Date'])
count=0
for i in all_stocks:
    holder = pdr.get_data_yahoo(i,'2019-07-22','2019-08-08')
    holder['Date'] = holder.index
    holder['Date'] = pd.to_datetime(holder['Date'], infer_datetime_format=True)
    holder = holder.reset_index(drop=True)
    holder['move']=(holder.Close-holder.Open)/holder.Open
    holder.drop(['High','Low','Open','Close','Volume','Adj Close'],axis=1,inplace=True)
    holder.rename(columns={'move':str(i)},inplace=True)
    daily_api_returns=daily_api_returns.merge(holder,left_on='Date',right_on='Date')
    count+=1
daily_api_returns=daily_api_returns.set_index('Date')
all_stocks=all_api_tweets.ticker.unique()
df_api=pd.DataFrame([])
for i in all_stocks:
    # isolating tweets about stock in question
    stock_tweets=all_api_tweets[all_api_tweets.ticker==i]
    stock_tweets.reset_index(inplace=True)
    for j in daily_api_returns.index:
        try:
            stock_move=daily_api_returns[i][j]
            end_time=j+np.timedelta64(9,"h")+np.timedelta64(30,'m')
            start_time=j-np.timedelta64(1,"D")+np.timedelta64(16,'h')
            stock_tweets_day=stock_tweets[(stock_tweets.date_time>start_time)&(stock_tweets.date_time<end_time)]
            df_api=df_api.append(pd.DataFrame({'stock':i,'trade_date':j,'stock_vader':stock_tweets_day.vader_score.mean(),
                                               'stock_weighted_vader':stock_tweets_day.weighted_vader.sum()/stock_tweets_day.followers.sum(),
                                               'stock_blob':stock_tweets_day.blob_score.mean(),
                                               'stock_weighted_blob':stock_tweets_day.weighted_blob.sum()/stock_tweets_day.followers.sum(),
                                               'num_stock_tweets':len(stock_tweets_day),'stock_tweets_reach':stock_tweets_day.followers.sum(),
                                               'stock_move':stock_move},index=[0]),ignore_index=True)
        except:
            print('fail',i,j)
df_api=df_api.dropna()
df_api.reset_index(drop=True,inplace=True)
df_api['stock_num_tweet_mean']=None
df_api['stock_num_tweet_std']=None
df_api['stock_tweet_sentiment_mean']=None
df_api['stock_tweet_sentiment_std']=None
sto=df_api.stock.unique()
for i in sto:
    temp=df_api[df_api.stock==i]
    num_mean=temp.num_stock_tweets.mean()
    num_std=temp.num_stock_tweets.std()
    sent_mean=temp.stock_weighted_vader.mean()
    sent_std=temp.stock_weighted_vader.std()
    df_api.loc[df_api['stock'].str.contains(i),'stock_num_tweet_mean']=num_mean
    df_api.loc[df_api['stock'].str.contains(i),'stock_num_tweet_std']=num_std
    df_api.loc[df_api['stock'].str.contains(i),'stock_tweet_sentiment_mean']=sent_mean
    df_api.loc[df_api['stock'].str.contains(i),'stock_tweet_sentiment_std']=sent_std
pickle_out=open("df_api.pickle","wb")
pickle.dump(df_api,pickle_out)
pickle_out.close()
pickle_out=open("daily_api_returns.pickle","wb")
pickle.dump(daily_api_returns,pickle_out)
pickle_out.close()

## Please continue to the notebook titled "Notebook_5_Live_Tweet_Analysis"